In [2]:
import pandas as pd
import asyncio
import openai


In [6]:
df = pd.read_json("data/dataset.jsonl", lines=True)

In [8]:
df

,text,len,source,tokens,prompt,gpt_query
0,"Что переживает человек, которого бросила Настя...",71,vk,29,"Расскажи анекдот про человека, которого бросил...",Составь точный текстовый запрос для анекдота в...
1,На ледовую арену поступил заказ. \nБыло заказа...,133,vk,69,Расскажи анекдот про заказ новых шайб на ледов...,Составь точный текстовый запрос для анекдота в...
2,"— Папа, а что у тебя за шапка на голове? \n— Э...",415,vk,225,Расскажи анекдот про защиту от жары в пустыне,Составь точный текстовый запрос для анекдота в...
3,"Идут инвалид и цыган по лесу... А нет, инвалид...",76,vk,40,Расскажи анекдот про инвалида и цыгана в лесу.,Составь точный текстовый запрос для анекдота в...
4,"Летят 2 кавказца на самолёте.\nСкорость 800, 8...",136,vk,75,"Расскажи анекдот про двух кавказцев, летящих н...",Составь точный текстовый запрос для анекдота в...
...,...,...,...,...,...,...
22276,Любовь - это оставлять некрасивую порцию еды с...,50,bad,27,Расскажи анекдот про любовь и некрасивую порци...,Составь точный текстовый запрос для анекдота в...
22277,- Почему в Афганистане студентов не учат вожде...,107,bad,55,Расскажи анекдот про студентов в Афганистане.,Составь точный текстовый запрос для анекдота в...
22278,- Уже неделю настраиваюсь заплатить за ЖКХ.- И...,79,bad,40,Расскажи анекдот про оплату ЖКХ.,Составь точный текстовый запрос для анекдота в...
22279,"Главное преимущество вина в коробке в том, что...",95,bad,42,Расскажи анекдот про вино в коробке и хомяка.,Составь точный текстовый запрос для анекдота в...


In [3]:
(~df["prompt"].isna()).sum()

14495

In [ ]:
!set -a
!source .env
!set +a

In [4]:
openai.api_key = os.environ.get("API_KEY")
client = openai.OpenAI(
    api_key=openai.api_key
)
aclient = openai.AsyncOpenAI(
    api_key=openai.api_key
)

In [5]:
def promp_gpt(question, model="gpt-3.5-turbo"):
    messages = [ 
        # {"role": "system", 
        # "content": "You are a helpfull assistant."},
        {"role": "user", "content": question}
    ] 
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
    )
    return chat_completion.choices[0].message.content 

async def a_promp_gpt(question, model="gpt-3.5-turbo"):
    messages = [ 
        # {"role": "system", 
        # "content": "You are a helpfull assistant."},
        {"role": "user", "content": question}
    ] 
    chat_completion = await aclient.chat.completions.create(
        messages=messages,
        model=model,
    )
    return chat_completion.choices[0].message.content 

In [6]:
async def a_croud_gpt(
    df,
    results,
    model="gpt-3.5-turbo",
    num_executors=50,
):
    tasks = asyncio.Queue()

    async def worker(tasks, results):
        while not tasks.empty():
            idx, question = await tasks.get()
            print(f"parsing idx {idx}")
            while True:
                try:
                    result = await a_promp_gpt(question, model=model)
                    results[idx] = result
                    df.loc[idx, "prompt"] = result
                    print(f"finished idx {idx}")
                    break
                except openai.RateLimitError: 
                    print(f"sleeping for {idx}")
                    await asyncio.sleep(60)
                except:
                    results[idx] = None
                    print(f"bad query for {idx}")
                    break

    # populate tasks queue
    for row in df[df["prompt"].isna()].iterrows():
        await tasks.put((row[0], row[1]["gpt_query"]))

    workers = [asyncio.create_task(worker(tasks, results))
        for _ in range(num_executors)
    ]
    await asyncio.gather(*workers)
    return results

In [7]:
template = """Составь точный текстовый запрос для анекдота в формате: "Расскажи анекдот про ...". Анекдот:

{anekdot}"""

In [8]:
# ["gpt-3.5-turbo-0125", "gpt-3.5-turbo"]

async def iter_save_pandas(df):
    while True:
        await asyncio.sleep(30)
        df.to_json("iter_save_dataset.jsonl", lines=True, orient="records")


model = "gpt-3.5-turbo"
results = [None] * len(df)
await asyncio.gather(
    a_croud_gpt(
        df, 
        results,
        model=model,
        num_executors=20,
    ),
    iter_save_pandas(df)
)

parsing idx 1967
parsing idx 8883
parsing idx 14478
parsing idx 14482
parsing idx 14489
parsing idx 14490
parsing idx 14494
parsing idx 14495
parsing idx 14497
parsing idx 14499
parsing idx 14500
parsing idx 14502
parsing idx 14505
parsing idx 14506
parsing idx 14509
parsing idx 14510
parsing idx 14511
parsing idx 14512
parsing idx 14513
parsing idx 14514
bad query for 1967
parsing idx 14515
finished idx 14490
parsing idx 14516
finished idx 14482
parsing idx 14517
finished idx 14513
parsing idx 14518
finished idx 14512
parsing idx 14519
finished idx 14506
parsing idx 14520
finished idx 14502
parsing idx 14521
finished idx 14495
parsing idx 14522
finished idx 14497
parsing idx 14523
finished idx 14511
parsing idx 14524
finished idx 14489
parsing idx 14525
finished idx 14499
parsing idx 14526
finished idx 14494
parsing idx 14527
finished idx 14514
parsing idx 14528
finished idx 14510
parsing idx 14529
finished idx 14500
parsing idx 14530
finished idx 14505
parsing idx 14531
finished idx 